# Light curve fitting

In [ ]:
from __future__ import print_function

import numpy as np
import sncosmo
import matplotlib as mpl

%matplotlib inline

mpl.rc('savefig', dpi=110.0)

### Example data

In [ ]:
data = sncosmo.read_lc('data/lc-SDSS19230.list', format='salt2')

In [ ]:
# data is an astropy Table
print(data)

In [ ]:
# Table metadata
print("z =", data.meta['Z_HELIO'])
print("mwebv =", data.meta['MWEBV'])

In [ ]:
# Quick visualization
sncosmo.plot_lc(data);

### Set up a model

In [ ]:
model1 = sncosmo.Model(source='hsiao',
                       effects=[sncosmo.F99Dust()],
                       effect_names=['mw'],
                       effect_frames=['obs'])

In [ ]:
print(model1)

In [ ]:
# fix some values
model1.set(mwebv=data.meta['MWEBV'], z=data.meta['Z_HELIO'])

In [ ]:
# fit the model to the data
result, model1 = sncosmo.fit_lc(data, model1, ['amplitude', 't0'])

In [ ]:
print(result)

In [ ]:
# access result components:
result.chisq, result.ndof

In [ ]:
sncosmo.plot_lc(data, model1, errors=result.errors);

### What's happening under the hood?

In [ ]:
# here is one row of the data table:
i = 50
print(data['Filter'][i], data['Date'][i], data['ZP'][i], data['MagSys'][i], '-->',
      data['Flux'][i], '+/-', data['Fluxerr'][i])

In [ ]:
# the model can predict the observed flux
model1.bandflux(data['Filter'][i], data['Date'][i], zp=data['ZP'][i], zpsys=data['MagSys'][i])

In [ ]:
# do it for all rows simultaneously:
predicted_flux = model1.bandflux(data['Filter'], data['Date'], zp=data['ZP'], zpsys=data['MagSys'])

sncosmo.fit_lc is using this function to define a likelihood or chisq, then passing that to an optimizer.

### A better model: include color

In [ ]:
model2 = sncosmo.Model(source='hsiao',
                       effects=[sncosmo.F99Dust(), sncosmo.F99Dust()],
                       effect_names=['mw', 'host'],
                       effect_frames=['obs', 'rest'])

In [ ]:
# fix known values
model2.set(mwebv=data.meta['MWEBV'], z=data.meta['Z_HELIO'])

In [ ]:
print(model2)

In [ ]:
# fit the model to the data
result2, model2 = sncosmo.fit_lc(data, model2, ['amplitude', 'hostebv', 't0'])

In [ ]:
print(result2)

To plot the data along with the best-fit model, set the model parameters to the best-fit values and plot.

In [ ]:
sncosmo.plot_lc(data, model=model2, errors=result2.errors);

### Yet another model

In [ ]:
model3 = sncosmo.Model(source='salt2',
                       effects=[sncosmo.F99Dust()],
                       effect_names=['mw'],
                       effect_frames=['obs'])

In [ ]:
# fix known values
model3.set(mwebv=data.meta['MWEBV'], z=data.meta['Z_HELIO'])

In [ ]:
# fit the model to the data
result3, model3 = sncosmo.fit_lc(data, model3, ['x0', 'x1', 'c', 't0'],
                                 bounds={'x1': (-3., 3.), 'c':(-0.3, 0.3)})

In [ ]:
print(result3)

In [ ]:
sncosmo.plot_lc(data, model=model3, errors=result3.errors);

### Why are x0 and amplitude so different?

In [ ]:
print(model1.get('amplitude'), " versus ", model3.get('x0'))

In [ ]:
model1.source_peakmag('bessellb', 'vega')

In [ ]:
model3.source_peakmag('bessellb', 'vega')

### Suppose we didn't know the redshift

In [ ]:
# fit the model to the data
result4, model3 = sncosmo.fit_lc(data, model3, ['x0', 'x1', 'c', 't0', 'z'],
                                 bounds={'x1': (-3., 3.), 'c':(-0.3, 0.3), 'z':(0.05, 0.45)})

In [ ]:
result4.ncall

In [ ]:
sncosmo.plot_lc(data, model=model3, errors=result4.errors);

## Sampling

In [ ]:
# takes about a minute
result, model3 = sncosmo.mcmc_lc(data, model3, ['x0', 'x1', 'c', 't0', 'z'],
                                 bounds={'x1': (-3., 3.), 'c':(-0.3, 0.3), 'z':(0.05, 0.45)},
                                 nwalkers=50, nburn=100, nsamples=400)

In [ ]:
print(result)

In [ ]:
import corner

In [ ]:
corner.corner(result.samples, bins=20, labels=result.vparam_names, show_titles=True);